In [1]:
import numpy as np
np.random.seed(1337)

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Lambda, Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout, Activation
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
from sklearn import metrics
from keras.regularizers import WeightRegularizer
import time

Using Theano backend.


In [2]:

dummy_empty = 2 ** 17

context_name_train = []
context_type_train = []
context_category_train = []
context_barea_train = []
context_address_train = []

ad_name_train = []
ad_type_train = []
ad_category_train = []
ad_barea_train = []
ad_address_train = []

y_train = []


with open('../../data/click-all-train') as f:
    for line in f:
        label, context_name, context_type, context_category, context_barea, context_address, ad_name, ad_type, ad_category, ad_barea, ad_address = line.split("\t")
        y_train.append(int(label))
        context_name_train.append(map(lambda x: int(x) if x else dummy_empty, context_name.strip().split(" ")))
        context_type_train.append(map(lambda x: int(x) if x else dummy_empty, context_type.strip().split(" ")))
        context_category_train.append(map(lambda x: int(x) if x else dummy_empty, context_category.strip().split(" ")))
        context_barea_train.append(map(lambda x:  int(x) if x else dummy_empty, context_barea.strip().split(" ")))
        context_address_train.append(map(lambda x:  int(x) if x else dummy_empty, context_address.strip().split(" ")))
        ad_name_train.append(map(lambda x: int(x) if x else dummy_empty, ad_name.strip().split(" ")))
        ad_type_train.append(map(lambda x: int(x) if x else dummy_empty, ad_type.strip().split(" ")))
        ad_category_train.append(map(lambda x: int(x) if x else dummy_empty, ad_category.strip().split(" ")))
        ad_barea_train.append(map(lambda x: int(x) if x else dummy_empty, ad_barea.strip().split(" ")))
        ad_address_train.append(map(lambda x: int(x) if x else dummy_empty, ad_address.strip().split(" ")))


context_name_test = []
context_type_test = []
context_category_test = []
context_barea_test = []
context_address_test = []


ad_name_test = []
ad_type_test = []
ad_category_test = []
ad_barea_test = []
ad_address_test = []


y_test = []
with open('../../data/click-all-test') as f:
    for line in f:
        label, context_name, context_type, context_category, context_barea, context_address, ad_name, ad_type, ad_category, ad_barea, ad_address = line.split("\t")
        y_test.append(int(label))
        context_name_test.append(map(lambda x: int(x) if x else dummy_empty, context_name.strip().split(" ")))
        context_type_test.append(map(lambda x:  int(x) if x else dummy_empty, context_type.strip().split(" ")))
        context_category_test.append(map(lambda x:  int(x) if x else dummy_empty, context_category.strip().split(" ")))
        context_barea_test.append(map(lambda x: int(x) if x else dummy_empty, context_barea.strip().split(" ")))
        context_address_test.append(map(lambda x:  int(x) if x else dummy_empty, context_address.strip().split(" ")))
        ad_name_test.append(map(lambda x:  int(x) if x else dummy_empty, ad_name.strip().split(" ")))
        ad_type_test.append(map(lambda x:  int(x) if x else dummy_empty, ad_type.strip().split(" ")))
        ad_category_test.append(map(lambda x:  int(x) if x else dummy_empty, ad_category.strip().split(" ")))
        ad_barea_test.append(map(lambda x:  int(x) if x else dummy_empty, ad_barea.strip().split(" ")))
        ad_address_test.append(map(lambda x:  int(x) if x else dummy_empty, ad_address.strip().split(" ")))

In [3]:

maxlen = 15


context_name_train = sequence.pad_sequences(context_name_train, maxlen=maxlen)
context_type_train = sequence.pad_sequences(context_type_train, maxlen=maxlen)
context_category_train = sequence.pad_sequences(context_category_train, maxlen=maxlen)
context_barea_train = sequence.pad_sequences(context_barea_train, maxlen=maxlen)
context_address_train = sequence.pad_sequences(context_address_train, maxlen=maxlen)

ad_name_train = sequence.pad_sequences(ad_name_train, maxlen=maxlen)
ad_type_train = sequence.pad_sequences(ad_type_train, maxlen=maxlen)
ad_category_train = sequence.pad_sequences(ad_category_train, maxlen=maxlen)
ad_barea_train = sequence.pad_sequences(ad_barea_train, maxlen=maxlen)
ad_address_train = sequence.pad_sequences(ad_address_train, maxlen=maxlen)


context_name_test = sequence.pad_sequences(context_name_test, maxlen=maxlen)
context_type_test = sequence.pad_sequences(context_type_test, maxlen=maxlen)
context_category_test = sequence.pad_sequences(context_category_test, maxlen=maxlen)
context_barea_test = sequence.pad_sequences(context_barea_test, maxlen=maxlen)
context_address_test = sequence.pad_sequences(context_address_test, maxlen=maxlen)

ad_name_test = sequence.pad_sequences(ad_name_test, maxlen=maxlen)
ad_type_test = sequence.pad_sequences(ad_type_test, maxlen=maxlen)
ad_category_test = sequence.pad_sequences(ad_category_test, maxlen=maxlen)
ad_barea_test = sequence.pad_sequences(ad_barea_test, maxlen=maxlen)
ad_address_test = sequence.pad_sequences(ad_address_test, maxlen=maxlen)

In [ ]:
def means(x):
    return K.mean(x, axis=1)

In [ ]:
from keras.layers import Merge

max_features = 2 ** 17 + 1
batch_size = 32
embedding_dims = 20


context_name = Sequential()
context_name.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform', dropout=0.5))
# context_name.add(Dropout(0.5))
context_name.add(Lambda(means, output_shape=(embedding_dims,)))

context_type = Sequential()
context_type.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform', dropout=0.5))
# context_type.add(Dropout(0.5))
context_type.add(Lambda(means, output_shape=(embedding_dims,)))

context_category = Sequential()
context_category.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform', dropout=0.5))
# context_category.add(Dropout(0.5))
context_category.add(Lambda(means, output_shape=(embedding_dims,)))

context_barea = Sequential()
context_barea.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform', dropout=0.5))
# context_barea.add(Dropout(0.5))
context_barea.add(Lambda(means, output_shape=(embedding_dims,)))

context_address = Sequential()
context_address.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform', dropout=0.5))
# context_address.add(Dropout(0.5))
context_address.add(Lambda(means, output_shape=(embedding_dims,)))

ad_name = Sequential()
ad_name.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform', dropout=0.5))
ad_name.add(Dropout(0.5))
ad_name.add(Lambda(means, output_shape=(embedding_dims,)))

ad_type = Sequential()
ad_type.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform', dropout=0.5))
ad_type.add(Dropout(0.5))
ad_type.add(Lambda(means, output_shape=(embedding_dims,)))

ad_category = Sequential()
ad_category.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform', dropout=0.5))
ad_category.add(Dropout(0.5))
ad_category.add(Lambda(means, output_shape=(embedding_dims,)))

ad_barea = Sequential()
ad_barea.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform', dropout=0.5))
ad_barea.add(Dropout(0.5))
ad_barea.add(Lambda(means, output_shape=(embedding_dims,)))

ad_address = Sequential()
ad_address.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform', dropout=0.5))
ad_address.add(Dropout(0.5))
ad_address.add(Lambda(means, output_shape=(embedding_dims,)))



merged = Merge([context_name, context_type, context_category, context_barea, context_address, \
               ad_name, ad_type, ad_category, ad_barea, ad_address], mode='concat')

final_model = Sequential()
final_model.add(merged)

final_model.add(Dense(20))
final_model.add(BatchNormalization())
final_model.add(LeakyReLU(alpha=.01))
final_model.add(Dropout(0.5))
final_model.add(Dense(1))
final_model.add(BatchNormalization())
final_model.add(Activation('sigmoid'))

final_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['fmeasure'])

model_path = ''
for i in range(10):
    if model_path != '':
        final_model.load_weights(model_path)
    model_path = '../../model/weights.epoch' + str(i) + "_" + str(int(time.time())) + '.hdf5'
    weight_save_callback = ModelCheckpoint(model_path, monitor='val_loss', verbose=0, save_best_only=False, mode='auto')
    final_model.fit([context_name_train, context_type_train, context_category_train, context_barea_train, context_address_train, \
                     ad_name_train, ad_type_train, ad_category_train, ad_barea_train, ad_address_train], y_train,  batch_size=batch_size, verbose=2, validation_data=([context_name_test, context_type_test, context_category_test, context_barea_test, context_address_test, \
                     ad_name_test, ad_type_test, ad_category_test, ad_barea_test, ad_address_test], y_test), nb_epoch=1, callbacks=[weight_save_callback])
    
    y_pred = final_model.predict([context_name_train, context_type_train, context_category_train, context_barea_train, context_address_train, \
                     ad_name_train, ad_type_train, ad_category_train, ad_barea_train, ad_address_train])[:, 0]
    fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred)
    print "train auc -> " + str(metrics.auc(fpr, tpr))
    
    y_pred = final_model.predict([context_name_test, context_type_test, context_category_test, context_barea_test, context_address_test, \
                     ad_name_test, ad_type_test, ad_category_test, ad_barea_test, ad_address_test])[:, 0]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    print "test auc -> " + str(metrics.auc(fpr, tpr))



Train on 1993177 samples, validate on 250766 samples
Epoch 1/1


In [ ]:
final_model.fit?

In [ ]:
# final_model.load_weights('../../model/weights.01-0.69.hdf5')
# final_model.fit([context_train, ad_train], y_train, verbose=2, batch_size=batch_size,validation_data=([context_test, ad_test], y_test), nb_epoch=2, callbacks=[weight_save_callback])

In [ ]:
# from keras.layers import Merge

# batch_size = 32
# embedding_dims = 100


# left_branch = Sequential()
# left_branch.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform'))
# # left_branch.add(BatchNormalization())
# left_branch.add(Dropout(0.5))
# left_branch.add(Lambda(means, output_shape=(embedding_dims,)))
# # left_branch.add(Dense(32, input_dim=embeding_dim))

# right_branch = Sequential()
# right_branch.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform'))
# # left_branch.add(BatchNormalization())
# left_branch.add(Dropout(0.5))
# right_branch.add(Lambda(means, output_shape=(embedding_dims,)))
# # right_branch.add(Dense(32, input_dim=embeding_dim))

# merged = Merge([left_branch, right_branch], mode='concat')

# final_model = Sequential()
# final_model.add(merged)

# final_model.add(Dense(20))
# final_model.add(BatchNormalization())
# final_model.add(LeakyReLU(alpha=.01))
# final_model.add(Dense(1))
# final_model.add(BatchNormalization())
# final_model.add(Activation('sigmoid'))

# final_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['fmeasure'])


# from keras.utils.visualize_util import plot
# plot(model, to_file='model.png')

In [ ]:
# # load model to predict
# model_path = '../../model/weights.epoch3_1481833706.hdf5'
# final_model.load_weights(model_path)

# y_pred = final_model.predict([context_train, ad_train])[:, 0]
# fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred)
# print "train auc -> " + str(metrics.auc(fpr, tpr))

# y_pred = final_model.predict([context_test, ad_test])[:, 0]
# fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
# print "test auc -> " + str(metrics.auc(fpr, tpr))

In [ ]:
# ! ls ../../modelweights.epoch3_1481833706.hdf5

In [ ]:
val = [1,2,3]
var = K.variable(value=val)

In [ ]:
K.int_shape(var)

In [ ]:
b = var * var

In [ ]:
b.eval()